<a href="https://colab.research.google.com/github/jsdysw/knn-classifier/blob/master/k_nn_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset analysis
feature vector X = {f1, f2, f3, f4, f5, f6}

class vector C = {"satisfied", "unsatisfied"}

In [18]:
import pandas as pd

# load satisfaction_data.csv from the github repository
url = 'https://raw.githubusercontent.com/jsdysw/knn-classifier/master/satisfaction_data.csv?token=GHSAT0AAAAAABQYSI53RANCS5CDDMW3Y3OIYTRKKOQ' 
dataset = pd.read_csv(url, names=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'class'])
print("Examples of dataset : \n", dataset[0:5])

# split data into train_data(9) and test_data(1)
test_data = dataset[0:18000]
train_data = dataset[18000:]

print("\nShape of the dataset : ", dataset.shape)
print("Shape of test dataset : ", test_data.shape)
print("Shape of train dataset : ", train_data.shape)


Examples of dataset : 
    f1  f2  f3      f4  f5  f6        class
0  40   2   1   86872  25   9  unsatisfied
1  40   2   1  259323  54  10    satisfied
2  40   2   1  256813  43  14    satisfied
3  70   2   1  118793  51  13    satisfied
4  40   2   2  398918  28   9  unsatisfied

Shape of the dataset :  (20000, 7)
Shape of test dataset :  (18000, 7)
Shape of train dataset :  (2000, 7)
